# 오늘은 LeNet 구조를 만들어봅시다


LeNet 구조는 CNN이며, 초기에 만들어진 모델입니다. 

2가지 모델(Sigmoid, ReLU)를 만들어 두 모델의 성능을 비교해봅시다.


## 1.우선 필요 라이브러리를 import 합니다.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import datasets
import torchvision.transforms as transforms

import torch.optim as optim

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

## 2. 딥러닝 모델을 설계할 때 활용하는 장비 확인

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', device)

Using PyTorch version: 1.11.0  Device: cpu


## 3. MNIST 데이터 다운로드 

 1. Training data와 Test data 분리하기
 
 2. Training data를 Training data 와 Validation data로 분리하기

In [5]:
BATCH_SIZE = 64

transform = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize((.5),(.5)),
     transforms.Resize((28,28))
    ])

train_data = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('./data', train=False, download=True, transform=transform)

train, val = torch.utils.data.random_split(train_data,[int(len(train_data)*0.8),len(train_data)-int(len(train_data)*0.8)])


train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=BATCH_SIZE, shuffle=False)
val_loader = torch.utils.data.DataLoader(dataset=val, batch_size=BATCH_SIZE, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)

## 4. torch.nn을 이용하여 모델-1 만들기

   1) 아래의 그림 중 LeNet 구조를 구현 할 것
   
   2) Sigmoid 활성화 함수를 이용할 것
   
   
![](Comparison_image_neural_networks.svg.png)

In [7]:
import torch.nn as nn

class Model_1(nn.Module):
    def __init__(self):
        super(Model_1, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=6, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(in_channels=6,out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(in_features=16*5*5,out_features=120)
        self.fc2 = nn.Linear(in_features=120,out_features=84)
        self.fc3 = nn.Linear(in_features=84,out_features=10)
        self.sigmoid = nn.Sigmoid()
        self.avgp = nn.AvgPool2d(kernel_size=2,stride=2)
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.sigmoid(x)
        x = self.avgp(x)
        x = self.conv2(x)
        x = self.sigmoid(x)
        x = self.avgp(x)
        x = torch.flatten(x,1)
        x = self.sigmoid(x)
        x = self.fc1(x)
        x = self.sigmoid(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        
        return x

## 5. torch.nn을 이용하여 모델-2 만들기

   LeNet 모델에서 ReLU 활성화 함수를 사용하시요

In [8]:
class Model_2(nn.Module):
    def __init__(self):
        super(Model_2, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=6, kernel_size=5,padding=2)
        self.conv2 = nn.Conv2d(in_channels=6,out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(in_features=5*5*16,out_features=120)
        self.fc2 = nn.Linear(in_features=120,out_features=84)
        self.fc3 = nn.Linear(in_features=84,out_features=10)
        self.relu = nn.ReLU()
        self.avgp = nn.AvgPool2d(kernel_size=2,stride=2)
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.avgp(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.avgp(x)
        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        
        return x

## 7. 학습 준비하기

1) 1 epoch를 학습할 수 있는 함수 만들기

2) Test와 Validation data의 정확도 계산할 수 있는 함수 만들기

In [9]:
def training_epoch(train_loader, network, loss_func, optimizer, epoch):
    train_losses = []
    train_correct = 0
    log_interval = 300
    
    for batch_idx, (image, label) in enumerate(train_loader):
        image, label = image.to(device), label.to(device)
        

        # 미분값의 초기화
        optimizer.zero_grad()

        # Forward propagration 계산하기.
        outputs = network(image)
        
        
        # Cross_entropy 함수를 적용하여 loss를 구하고 저장하기
        loss = loss_func(outputs,label)
        train_losses.append(loss.item())

        # training accuracy 정확도 구하기 위해 맞는 샘플 개수 세기
        pred = torch.max(outputs,1)[1]
        train_correct += pred.eq(label).sum()

        # Gradinet 구하기
        loss.backward()

        # weight값 update 하기
        optimizer.step()

        # 학습 상황 출력
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.2f}%)]\tLoss: {:.6f}'
                  .format(epoch, batch_idx * len(label), len(train_loader.dataset),100. * batch_idx / len(train_loader),
                          loss.item()))
            
    return train_losses, train_correct

In [10]:
def test_epoch(test_loader, network, loss_func, val = False):
    correct = 0
    
    test_losses = []
    
    with torch.no_grad():
        for batch_idx, (image, label) in enumerate(test_loader):
            image, label = image.to(device), label.to(device)
            

            # Forward propagration 계산하기.
            outputs = network(image)

            # Cross_entropy 함수를 적용하여 loss를 구하기
            loss = loss_func(outputs,label)
            test_losses.append(loss.item())

            # Batch 별로 정확도 구하기
            pred = torch.max(outputs,1)[1]
            correct += pred.eq(label).sum()

        # 전체 정확도 구하기
        test_accuracy = 100. * correct / len(test_loader.dataset)

        #중간결과 출력
        if val is True:
                print('Validation set: Accuracy: {}/{} ({:.2f}%)\n'
              .format(correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))
        
        else:
            print('Test set: Accuracy: {}/{} ({:.2f}%)\n'
                  .format(correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))
        
    return test_losses, test_accuracy


## 8. 위 정의된 함수로 학습 함수 만들기

Adam Optimizer를 사용하여 학습시키기

In [11]:
def training(network, learning_rate = 0.001):
    
    epoches = 15
    
    cls_loss = nn.CrossEntropyLoss()
    optimizer = optim.Adam(network.parameters(),lr=learning_rate)
    
    train_losses_per_epoch = []
    test_losses_per_epoch = []
    
    train_accuracies = []
    test_accuracies = []
    
    
    for epoch in range(epoches):
                
        # 모델를 학습 중이라고 선언하기
        network.train()
        
        train_losses, train_correct = training_epoch(train_loader,network,cls_loss,optimizer, epoch)
        
        # epoch 별로 loss 평균값, 정확도 구하기
        average_loss = np.mean(train_losses)
        train_losses_per_epoch.append(average_loss)
        
        train_accuracy = train_correct / len(train_loader.dataset) * 100
        train_accuracies.append(train_accuracy)
        
        # epoch 별로 정확도 출력
        print('\nTraining set: Accuracy: {}/{} ({:.2f}%)'
              .format(train_correct, len(train_loader.dataset),100. * train_correct / len(train_loader.dataset)))

        
        ### 학습 중에 test 결과 보기
        
        # 모델 test 중인 것을 선언하기
        network.eval()
        
        correct = 0
        with torch.no_grad():
            test_losses, test_accuracy = test_epoch(val_loader, network, cls_loss, True)

        test_losses_per_epoch.append(np.mean(test_losses))
        test_accuracies.append(test_accuracy)
        
    with torch.no_grad():
        test_losses, test_accuracy = test_epoch(test_loader, network, cls_loss, False)
        
    return train_losses_per_epoch, test_losses_per_epoch, train_accuracies, test_accuracies


In [12]:
network = Model_1().to(device)
rlt_const = training(network)

Train Epoch: 0 [0/48000 (0.00%)]	Loss: 2.304197
Train Epoch: 0 [19200/48000 (40.00%)]	Loss: 2.296089
Train Epoch: 0 [38400/48000 (80.00%)]	Loss: 1.976232

Training set: Accuracy: 14389/48000 (29.98%)
Validation set: Accuracy: 8292/12000 (69.10%)

Train Epoch: 1 [0/48000 (0.00%)]	Loss: 1.879800
Train Epoch: 1 [19200/48000 (40.00%)]	Loss: 1.762307
Train Epoch: 1 [38400/48000 (80.00%)]	Loss: 1.738256

Training set: Accuracy: 32573/48000 (67.86%)
Validation set: Accuracy: 7993/12000 (66.61%)

Train Epoch: 2 [0/48000 (0.00%)]	Loss: 1.747828
Train Epoch: 2 [19200/48000 (40.00%)]	Loss: 1.664097
Train Epoch: 2 [38400/48000 (80.00%)]	Loss: 1.667690

Training set: Accuracy: 35049/48000 (73.02%)
Validation set: Accuracy: 9315/12000 (77.62%)

Train Epoch: 3 [0/48000 (0.00%)]	Loss: 1.668353
Train Epoch: 3 [19200/48000 (40.00%)]	Loss: 1.607952
Train Epoch: 3 [38400/48000 (80.00%)]	Loss: 1.616962

Training set: Accuracy: 38316/48000 (79.82%)
Validation set: Accuracy: 9749/12000 (81.24%)

Train Epoch:

In [13]:
network = Model_2().to(device)
rlt_const = training(network)

Train Epoch: 0 [0/48000 (0.00%)]	Loss: 2.310854
Train Epoch: 0 [19200/48000 (40.00%)]	Loss: 0.914681
Train Epoch: 0 [38400/48000 (80.00%)]	Loss: 1.172450

Training set: Accuracy: 26496/48000 (55.20%)
Validation set: Accuracy: 6930/12000 (57.75%)

Train Epoch: 1 [0/48000 (0.00%)]	Loss: 1.323341
Train Epoch: 1 [19200/48000 (40.00%)]	Loss: 0.774044
Train Epoch: 1 [38400/48000 (80.00%)]	Loss: 1.114361

Training set: Accuracy: 28018/48000 (58.37%)
Validation set: Accuracy: 6979/12000 (58.16%)

Train Epoch: 2 [0/48000 (0.00%)]	Loss: 1.236820
Train Epoch: 2 [19200/48000 (40.00%)]	Loss: 0.765088
Train Epoch: 2 [38400/48000 (80.00%)]	Loss: 1.114369

Training set: Accuracy: 28190/48000 (58.73%)
Validation set: Accuracy: 6992/12000 (58.27%)

Train Epoch: 3 [0/48000 (0.00%)]	Loss: 1.205572
Train Epoch: 3 [19200/48000 (40.00%)]	Loss: 0.762643
Train Epoch: 3 [38400/48000 (80.00%)]	Loss: 1.101266

Training set: Accuracy: 28265/48000 (58.89%)
Validation set: Accuracy: 7003/12000 (58.36%)

Train Epoch:

## 9. 두모델의 성능을 비교하시오

정답) 
model2가 loss값은 작지만, 최종 정확도에서 59.09%로 나와, 88.90%가 나온 model1보다는 성능이 부족하다고 할 수 있다.
따라서 LeNet에서는 활성함수를 sigmoid를 사용하여 모델의 성능을 높일 수 있다.